## Kidney disease classification 

1. Github Repository Setup
2. Project Template Creation
3. project Setup & Requirements installation
4. Logging ,Utils & Exception Module
5. Project workflows
6. All components Notebook Experiments
7. All components Modular code implementation
8. training Pipeline
9. MLflow (MLops Tool) -- For  Experiments tracking & model Registration
10. DVC(MLops Tool)  --  For Pipeline Tracking & implementation
11. Prediction pipeline & user App creation
12. Docker
13. Final CI/CD deployment on AWS

## What is kidney tumor ?
* A kidney tumor is an abnormal growth or mass that develops in the kidney. Kidney tumors can be benign (non-cancerous) or malignant (cancerous)

# 1. Github Repository Setup

# 2. Project Template Creation

# 1. step  create `template.py` file in vscode

### template.py

In [ ]:
import os
from pathlib import Path
import logging

#logging string
logging.basicConfig(level=logging.INFO, format='[%(asctime)s]: %(message)s:')

project_name = 'cnnClassifier'

list_of_files = [
    ".github/workflows/.gitkeep",
    f"src/{project_name}/__init__.py",
    f"src/{project_name}/components/__init__.py",
    f"src/{project_name}/utils/__init__.py",
    f"src/{project_name}/config/__init__.py",
    f"src/{project_name}/config/configuration.py",
    f"src/{project_name}/pipeline/__init__.py",
    f"src/{project_name}/entity/__init__.py",
    f"src/{project_name}/constants/__init__.py",
    "config/config.yaml",
    "dvc.yaml",
    "params.yaml",
    "requirements.txt",
    "setup.py",
    "research/trials.ipynb",
    "templates/index.html"


]


for filepath in list_of_files:
    filepath = Path(filepath)
    filedir, filename = os.path.split(filepath)


    if filedir !="":
        os.makedirs(filedir, exist_ok=True)
        logging.info(f"Creating directory; {filedir} for the file: {filename}")

    if (not os.path.exists(filepath)) or (os.path.getsize(filepath) == 0):
        with open(filepath, "w") as f:
            pass
            logging.info(f"Creating empty file: {filepath}")


    else:
        logging.info(f"{filename} is already exists")

## run command 'python template.py' in cmd

## requirements.txt

In [ ]:
tensorflow==2.12.0
pandas 
dvc
mlflow==2.2.2
notebook
numpy
matplotlib
seaborn
python-box==6.0.2
pyYAML
tqdm
ensure==1.0.2
joblib
types-PyYAML
scipy
Flask
Flask-Cors
gdown
-e .

`setup.py`

In [ ]:
import setuptools

with open("README.md", "r", encoding="utf-8") as f:
    long_description = f.read()


__version__ = "0.0.0"

REPO_NAME = "Kidney-disease-classification"
AUTHOR_USER_NAME = "hellovisha"
SRC_REPO = "cnnClassifier"
AUTHOR_EMAIL = "vishalsinghrawat505@gmail.com"


setuptools.setup(
    name=SRC_REPO,
    version=__version__,
    author=AUTHOR_USER_NAME,
    author_email=AUTHOR_EMAIL,
    description="A small python package for CNN app",
    long_description=long_description,
    long_description_content="text/markdown",
    url=f"https://github.com/{AUTHOR_USER_NAME}/{REPO_NAME}",
    project_urls={
        "Bug Tracker": f"https://github.com/{AUTHOR_USER_NAME}/{REPO_NAME}/issues",
    },
    package_dir={"": "src"},
    packages=setuptools.find_packages(where="src")
)

# create virtualenv


In [ ]:
conda create -n myenv python=3.8
conda activate myenv
python --version


then install `requirements.txt`

In [ ]:
pip install -r requirements.txt

# 4 Logging, Exception & Utils Modules

## Logging

### src/__init__.py

In [ ]:
import os  # Importing the os module to interact with the operating system
import sys  # Importing the sys module to interact with the Python runtime environment
import logging  # Importing the logging module to handle logging

# Define the logging format string
logging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"

# Define the directory where logs will be stored
log_dir = "logs"

# Define the full file path for the log file
log_filepath = os.path.join(log_dir, "running_logs.log")

# Create the log directory if it doesn't exist
os.makedirs(log_dir, exist_ok=True)

# Configure the logging settings
logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format=logging_str,  # Set the logging format

    # Define the handlers for logging
    handlers=[
        logging.FileHandler(log_filepath),  # Log to a file
        logging.StreamHandler(sys.stdout)  # Also log to standard output (console)
    ]
)

# Create a logger with the name 'cnnClassifierLogger'
logger = logging.getLogger("cnnClassifierLogger")


## create file in project folder main.py

## main.py

In [ ]:
from src.cnnClassifier import logger



logger.info("Welcome to the custom log") 


# Exception 

### create python file inside utils folder 

# common.py

In [ ]:
import os  # Module to interact with the operating system
from box.exceptions import BoxValueError  # Exception handling from the Box library
import yaml  # Module to work with YAML files
from cnnClassifier import logger  # Importing the logger from the cnnClassifier module
import json  # Module to work with JSON files
import joblib  # Module to save and load Python objects as binary files
from ensure import ensure_annotations  # Decorator to enforce type annotations
from box import ConfigBox  # Class to convert dictionaries into objects
from pathlib import Path  # Module to work with filesystem paths
from typing import Any  # For type hinting
import base64  # Module to encode and decode data with Base64

@ensure_annotations
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """Reads YAML file and returns its contents as a ConfigBox.

    Args:
        path_to_yaml (Path): Path to the YAML file.

    Raises:
        ValueError: If the YAML file is empty.
        Exception: For other exceptions.

    Returns:
        ConfigBox: Content of the YAML file as a ConfigBox object.
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            logger.info(f"YAML file: {path_to_yaml} loaded successfully")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("YAML file is empty")
    except Exception as e:
        raise e


@ensure_annotations
def create_directories(path_to_directories: list, verbose=True):
    """Create directories specified in the list.

    Args:
        path_to_directories (list): List of paths to directories to create.
        verbose (bool, optional): If True, log the directory creation. Defaults to True.
    """
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
        if verbose:
            logger.info(f"Created directory at: {path}")



@ensure_annotations
def save_json(path: Path, data: dict):
    """Save dictionary data to a JSON file.

    Args:
        path (Path): Path to the JSON file.
        data (dict): Data to be saved in the JSON file.
    """
    with open(path, "w") as f:
        json.dump(data, f, indent=4)
    logger.info(f"JSON file saved at: {path}")


@ensure_annotations
def load_json(path: Path) -> ConfigBox:
    """Load JSON file data.

    Args:
        path (Path): Path to the JSON file.

    Returns:
        ConfigBox: Data as class attributes instead of a dictionary.
    """
    with open(path) as f:
        content = json.load(f)
    logger.info(f"JSON file loaded successfully from: {path}")
    return ConfigBox(content)


@ensure_annotations
def save_bin(data: Any, path: Path):
    """Save data to a binary file.

    Args:
        data (Any): Data to be saved as a binary file.
        path (Path): Path to the binary file.
    """
    joblib.dump(value=data, filename=path)
    logger.info(f"Binary file saved at: {path}")

@ensure_annotations
def load_bin(path: Path) -> Any:
    """Load data from a binary file.

    Args:
        path (Path): Path to the binary file.

    Returns:
        Any: Object stored in the file.
    """
    data = joblib.load(path)
    logger.info(f"Binary file loaded from: {path}")
    return data

@ensure_annotations
def get_size(path: Path) -> str:
    """Get the size of the file in KB.

    Args:
        path (Path): Path of the file.

    Returns:
        str: Size of the file in KB.
    """
    size_in_kb = round(os.path.getsize(path) / 1024)
    return f"~ {size_in_kb} KB"